In [1]:
import os
import pandas as pd
import numpy as np

import polars as pl

In [15]:
data_dir = "~/siads696/data"

In [16]:
df_pl_img_embed = pl.read_csv(os.path.join(data_dir, "tl_features.csv"))
print(df_pl_img_embed)

shape: (13_200, 3_088)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Metadata_ ┆ Metadata_ ┆ Metadata_ ┆ Metadata_ ┆ … ┆ Z1020_Act ┆ Z1021_Act ┆ Z1022_Act ┆ Z1023_Ac │
│ ID        ┆ TableNumb ┆ ImageNumb ┆ Plate_DAP ┆   ┆ in        ┆ in        ┆ in        ┆ tin      │
│ ---       ┆ er        ┆ er        ┆ I         ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│           ┆ i64       ┆ i64       ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 4_233     ┆ 4         ┆ 233       ┆ Week4_274 ┆ … ┆ -0.321486 ┆ 1.2154524 ┆ -0.089709 ┆ -0.41329 │
│           ┆           ┆           ┆ 81        ┆   ┆           ┆           ┆           ┆ 8        │
│ 4_234     ┆ 4         ┆ 234       ┆ Week4_274 ┆ … ┆ -0.629341 ┆ 2.

In [17]:
df_pl_img_embed.select(
    pl.col("^Metadata.+$")
).columns

['Metadata_ID',
 'Metadata_TableNumber',
 'Metadata_ImageNumber',
 'Metadata_Plate_DAPI',
 'Metadata_Well_DAPI',
 'Metadata_Replicate',
 'Metadata_Compound',
 'Metadata_Concentration',
 'Metadata_SMILES',
 'Metadata_MoA']

In [18]:
plate_no = df_pl_img_embed.select(
    pl.col("Metadata_Plate_DAPI").str.extract(r".+_(\d+)", group_index=1).cast(pl.Int64).alias("Metadata_PlateNumber"),
)

In [19]:
df_pl_img_embed = pl.concat(
    [
        plate_no,
        df_pl_img_embed,
    ],
    how="horizontal",
)
print(df_pl_img_embed.select(pl.col("Metadata_PlateNumber", "Metadata_Plate_DAPI")))

shape: (13_200, 2)
┌──────────────────────┬─────────────────────┐
│ Metadata_PlateNumber ┆ Metadata_Plate_DAPI │
│ ---                  ┆ ---                 │
│ i64                  ┆ str                 │
╞══════════════════════╪═════════════════════╡
│ 27481                ┆ Week4_27481         │
│ 27481                ┆ Week4_27481         │
│ 27481                ┆ Week4_27481         │
│ 27481                ┆ Week4_27481         │
│ 27521                ┆ Week4_27521         │
│ …                    ┆ …                   │
│ 39301                ┆ Week9_39301         │
│ 39301                ┆ Week9_39301         │
│ 39301                ┆ Week9_39301         │
│ 39301                ┆ Week9_39301         │
│ 39301                ┆ Week9_39301         │
└──────────────────────┴─────────────────────┘


In [20]:
out = df_pl_img_embed.select(
    pl.col("*").exclude("Image_FileName_DAPI",
                        "Image_PathName_DAPI",
                        "Image_FileName_Tubulin",
                        "Image_PathName_Tubulin",
                        "Image_FileName_Actin",
                        "Image_PathName_Actin",
                        "Metadata_Plate_DAPI",
                        "Metadata_Well_DAPI",
                        )
                        
)
print(out)

shape: (13_200, 3_081)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Metadata_ ┆ Metadata_ ┆ Metadata_ ┆ Metadata_ ┆ … ┆ Z1020_Act ┆ Z1021_Act ┆ Z1022_Act ┆ Z1023_Ac │
│ PlateNumb ┆ ID        ┆ TableNumb ┆ ImageNumb ┆   ┆ in        ┆ in        ┆ in        ┆ tin      │
│ er        ┆ ---       ┆ er        ┆ er        ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ str       ┆ ---       ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│ i64       ┆           ┆ i64       ┆ i64       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 27481     ┆ 4_233     ┆ 4         ┆ 233       ┆ … ┆ -0.321486 ┆ 1.2154524 ┆ -0.089709 ┆ -0.41329 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 8        │
│ 27481     ┆ 4_234     ┆ 4         ┆ 234       ┆ … ┆ -0.629341 ┆ 2.

In [21]:
out.write_csv(os.path.join(data_dir, "bbbc021_image_embed_compact.csv"))